In [1]:
import numpy as np
from numba import njit, float64, int64
import time

## Naive Python code (Univariate Normal)

In [2]:
def normal_ll(X, mu, sigma):
    return np.exp(-(X-mu)**2 / (2 * sigma)) / np.sqrt(2*np.pi*sigma)

In [3]:
def GMM_EM(X, mu, sigma, max_iter, tau, q, tol = 1e-15):
    K = len(mu)
    n = len(X)
    
    for iteration in range(max_iter):
        for k in range(K):
            ll = normal_ll(X, mu[k], sigma[k])
            q[:, k ] = tau[k] * ll
        
        for i in range(n):
            q[i, :] /= np.sum(q[i, :])
        
        mu_before = mu
        sigma_before = sigma
        tau_before = tau
        
        for k in range(K):
            q_k = np.sum(q[:, k])
            mu[k] = np.sum(q[:, k] * X) / q_k
            sigma[k] = np.sum(q[:, k] * (X - mu[k])**2) / q_k
            tau[k]  = q_k / n
            
        mu_diff = np.max(np.abs(mu - mu_before))
        sigma_diff = np.max(np.abs(sigma-sigma_before))
        tau_diff = np.max(np.abs(tau - tau_before))
        
        diff = np.max(np.array([np.abs(mu_diff), np.abs(sigma_diff), np.abs(tau_diff)]))
        
        if ( (iteration > 1) & (diff < tol)): break
        
    return mu, sigma, tau, iteration

## Numba Python code (Univariate Normal)

In [4]:
@njit('float64[:](float64[:],float64,float64)')
def normal_ll_njit(X, mu, sigma):
    return np.exp(-(X-mu)**2 / (2 * sigma)) / np.sqrt(2*np.pi*sigma)

In [5]:
@njit('float64(float64[:])')
def nb_sum(x):
    res = 0.0
    for i in range(x.shape[0]):
        res += x[i]
    return res

In [6]:
from numba import types

In [7]:
r_sig = types.Tuple([float64[:],float64[:],float64[:],int64])
sig = r_sig(float64[:],float64[:],float64[:],int64,float64[:],float64[:,:],float64)

In [8]:
@njit(sig)
def GMM_EM_njit(X, mu, sigma, max_iter, tau, q, tol = 1e-15):
    K = len(mu)
    n = len(X)
    
    for iteration in range(max_iter):
        for k in range(K):
            ll = normal_ll_njit(X, mu[k], sigma[k])
            q[:, k ] = tau[k] * ll
        
        for i in range(n):
            q[i, :] /= nb_sum(q[i, :])
        
        mu_before = mu
        sigma_before = sigma
        tau_before = tau
        
        for k in range(K):
            q_k = nb_sum(q[:, k])
            mu[k] = nb_sum(q[:, k] * X) / q_k
            sigma[k] = nb_sum(q[:, k] * (X - mu[k])**2) / q_k
            tau[k]  = q_k / n
            
        mu_diff = np.max(np.abs(mu - mu_before))
        sigma_diff = np.max(np.abs(sigma-sigma_before))
        tau_diff = np.max(np.abs(tau - tau_before))
        
        diff = np.max(np.array([np.abs(mu_diff), np.abs(sigma_diff), np.abs(tau_diff)]))
        
        if ( (iteration > 1) & (diff < tol)): break
        
    return mu, sigma, tau, iteration

In [9]:
np.random.seed(42)
time_list_naive = []

for i in range(10):
    
    X1 = np.random.normal(loc = 20., scale = 3.1, size= 200) 
    X2 = np.random.normal(loc = 3., scale = 2.3, size= 200) 
    X3 = np.random.normal(loc = -5., scale = 1.4, size= 200) 
    X_tot = np.hstack((X1,X2,X3)).flatten()
    
    mu = np.array([15.,6.,-7.])
    tau = np.array([1/3,1/3,1/3])
    sigma = np.array([8.,3.5,1.3])
    q = np.zeros((len(X_tot),3))
    
    t1 = time.time()
    GMM_EM(X_tot, mu, sigma, 10000, tau, q)
    t2 = time.time()
    
    time_list_naive.append(t2-t1)

In [10]:
np.random.seed(42)
time_list_njit = []

for i in range(10):
    
    X1 = np.random.normal(loc = 20., scale = 3.1, size= 200) 
    X2 = np.random.normal(loc = 3., scale = 2.3, size= 200) 
    X3 = np.random.normal(loc = -5., scale = 1.4, size= 200) 
    X_tot = np.hstack((X1,X2,X3)).flatten()
    
    mu = np.array([15.,6.,-7.])
    tau = np.array([1/3,1/3,1/3])
    sigma = np.array([8.,3.5,1.3])
    q = np.zeros((len(X_tot),3))
    
    t1 = time.time()
    GMM_EM_njit(X_tot, mu, sigma, 10000, tau, q,1e-15)
    t2 = time.time()
    
    time_list_njit.append(t2-t1)

In [11]:
print(np.mean(time_list_naive))
print(time_list_njit[0])
print(np.mean(time_list_njit[1:]))

0.010036802291870118
0.0003464221954345703
0.00015248192681206597
